# Setting Up

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install loguru coloredlogs matplotlib pandas loguru boto3 opencv-python bounding-box numpy Pillow imgaug

!pip install matplotlib numpy opencv-python Pillow PyYAML requests scipy torch torchvision tqdm tensorboard pandas seaborn thop

     |████████████████████████████████| 58 kB 4.4 MB/s 
     |████████████████████████████████| 46 kB 3.5 MB/s 
     |████████████████████████████████| 132 kB 15.4 MB/s 
     |████████████████████████████████| 164 kB 38.8 MB/s 
     |████████████████████████████████| 86 kB 5.9 MB/s 
     |████████████████████████████████| 8.6 MB 44.9 MB/s 
     |████████████████████████████████| 79 kB 7.7 MB/s 
     |████████████████████████████████| 138 kB 52.6 MB/s 
  Created wheel for bounding-box: filename=bounding_box-0.1.3-py3-none-any.whl size=163935 sha256=9ded6405778e37e702b8fd6a974d15e4f28937e1f5c4e98dda6c7eef2e5a6eb1
  Stored in directory: /root/.cache/pip/wheels/ac/43/da/4b01c5ebbd32a9814495d2cc81e07c95e13aa210aa3cd860b0
Successfully built bounding-box
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packa

In [4]:
import json
import random
import os, sys
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"]=25,25
from random import randrange, choice
from IPython.display import display, clear_output, HTML
from loguru import logger
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

#homeDir = "D:\\FYP\\"
#homeDir = "A:\\University\\Final-Year-Project-AI\\"
homeDir = "/content/drive/MyDrive/Final-Year-Project-AI/"

In [6]:
sys.path.append(os.path.dirname(os.path.realpath(os.getcwd())))
sys.path.append(os.path.dirname(os.path.realpath(os.getcwd())) + "/core")

os.chdir(homeDir + "object-sense-avoid-ai/core/")
# Install the required package
!pip install -r ../requirements.txt

from core.dataset import Dataset 

# Clearing the output
clear_output()

In [7]:
from core.helper import initialise, load_data

# Initialize the enviroment
initialise(42)

os.chdir(homeDir + "object-sense-avoid-ai/notebooks/")

print(os.getcwd())
#dataset = load_data()
notebook_path = os.path.dirname(os.path.realpath("__file__")) + "/data"
local_path = notebook_path + '/part1'
s3_path = 's3://airborne-obj-detection-challenge-training/part1/'
dataset = Dataset(local_path, s3_path, partial=True, prefix="part1")
local_path = notebook_path + '/part2'
s3_path = 's3://airborne-obj-detection-challenge-training/part2/'
dataset.add(local_path, s3_path, prefix="part2")
clear_output()

# Print out the dataset
print(dataset)

Dataset(num_flights=1698)


## Create Training Dataset for Yolo models

In [8]:
# Warning: Clone only if base YOLOv5 repo isnt downloaded (in notebooks folder)
# !git clone https://github.com/ultralytics/yolov5
# os.chdir("yolov5")

os.chdir(homeDir + "object-sense-avoid-ai/notebooks/yolov5")

# Install the required dependences
%pip install -r requirements.txt

# Import Pytorch
import torch
from IPython.display import Image, clear_output

# Clear output console
clear_output()

# Print out the working enviroment
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Setup complete. Using torch 1.10.0+cu111 (Tesla K80)


In [9]:
print(os.getcwd())

os.chdir(homeDir + "object-sense-avoid-ai/data")

flightBatchSize = 0;
flight_ids = dataset.get_flight_ids()
# while flightBatchSize < 50:
#     #for flight in flight_ids:
#     currentFlight = dataset.get_flight(flight_ids[flightBatchSize])
#     if currentFlight.num_airborne_objs >= 2:
#       logger.info("Downloading Flight {}: {}", flightBatchSize, currentFlight)
#       currentFlight.download()
#     flightBatchSize += 1


/content/drive/MyDrive/Final-Year-Project-AI/object-sense-avoid-ai/notebooks/yolov5


In [ ]:
# os.chdir(homeDir + "object-sense-avoid-ai/core")
# from core.yolo_data import Yolo_Data
# os.chdir(homeDir + "object-sense-avoid-ai/notebooks/yolov5/data")

# # Create an instance
# yolo_data = Yolo_Data()
# # Generate training datas and labels
# yolo_data.generate_data(dataset)

Iterating through ./notebooks/data/part1/Images/
Generating train.txt
Generating test.txt
Generating original_paths.txt
Finish Generating !!! Total of 1330 images and 2959 objects


In [ ]:
# os.chdir(homeDir + "object-sense-avoid-ai\\notebooks\\yolov5\\data")
# if not os.path.exists("object-sense-avoid-ai/notebooks/yolov5/data/obj"):
#     open(homeDir + "object-sense-avoid-ai/notebooks/yolov5/data/obj", 'a').close()

In [19]:
os.chdir(homeDir + "object-sense-avoid-ai/notebooks/yolov5/data")

In [20]:
%%file aot.yaml
path: 
train: train.txt
val: test.txt
test: test.txt
nc: 3
names: ['Airplane', 'Helicopter', 'Airborne']

Overwriting aot.yaml


In [ ]:
# %%file aot.yaml
# path: D:/FYP/object-sense-avoid-ai/notebooks/yolov5/data/
# train: train.txt
# val: train.txt
# test: test.txt
# nc: 6
# names: ['Airplane', 'Helicopter', 'Drone', 'Airborne', 'Flock', 'Bird']

## Selecting a model & training model on AOT Dataset

In [11]:
# https://wandb.ai
# Install wandb
!pip install wandb
# Clear the output
clear_output()

In [12]:
#01d19932ed834c82bc20737a8f396c1d6649a108
!wandb login

wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
# http://localhost:6006/
# Tensorboard (optional)
%load_ext tensorboard
# Access the training directory
%tensorboard --logdir runs/train

In [ ]:
os.chdir(homeDir + "object-sense-avoid-ai/notebooks/yolov5/data")
!python ../train.py --img 1600 --batch 8 --epochs 10 --data aot.yaml --weights /content/drive/MyDrive/Final-Year-Project-AI/object-sense-avoid-ai/notebooks/yolov5/runs/train/exp12/weights/best.pt

wandb: Currently logged in as: pgosw (use `wandb login --relogin` to force relogin)
train: weights=/content/drive/MyDrive/Final-Year-Project-AI/object-sense-avoid-ai/notebooks/yolov5/runs/train/exp12/weights/best.pt, cfg=, data=aot.yaml, hyp=../data/hyps/hyp.scratch.yaml, epochs=10, batch_size=8, imgsz=1600, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=../runs/train, name=exp, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: skipping check (not a git repository), for updates see https://github.com/ultralytics/yolov5
YOLOv5 🚀 8b9537a torch 1.10.0+cu111 CUDA:0 (Tesla K80, 11441MiB)

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warm

In [14]:
# Load our model
model = torch.hub.load('./', 'custom', path='./runs/train/exp9/weights/best.pt', source='local')
# Cleaning the output
clear_output()
# Image Path
img = '/content/drive/MyDrive/Final-Year-Project-AI/object-sense-avoid-ai/notebooks/testImages/15553341149037764548af6dc9c2d794ea683e9554a0e728ca3.png'
# Inference
results = model(img)

FileNotFoundError: ignored

In [ ]:
# Print out the prediction
results.print()  
# results.save()  

# Return the Tensorflow format
results.xyxy[0]

# Show the labeled image
results.show()

# Return the prediction dataframe
results.pandas().xyxy[0]

image 1/1: 2048x2448 1 Airplane, 1 Helicopter
Speed: 118.5ms pre-process, 32.1ms inference, 2.3ms NMS per image at shape (1, 3, 544, 640)


,xmin,ymin,xmax,ymax,confidence,class,name
0,2012.078125,958.723083,2055.644287,979.833557,0.708333,0,Airplane
1,2014.387695,956.110901,2051.807617,981.800781,0.625428,1,Helicopter


In [25]:
!python /content/drive/MyDrive/Final-Year-Project-AI/object-sense-avoid-ai/notebooks/yolov5/val.py --weights /content/drive/MyDrive/Final-Year-Project-AI/object-sense-avoid-ai/notebooks/yolov5/runs/train/exp13/weights/best.pt /content/drive/MyDrive/Final-Year-Project-AI/object-sense-avoid-ai/notebooks/yolov5/runs/train/exp9/weights/best.pt --img 1600 --data /content/drive/MyDrive/Final-Year-Project-AI/object-sense-avoid-ai/notebooks/yolov5/data/aot.yaml --augment

val: data=/content/drive/MyDrive/Final-Year-Project-AI/object-sense-avoid-ai/notebooks/yolov5/data/aot.yaml, weights=['/content/drive/MyDrive/Final-Year-Project-AI/object-sense-avoid-ai/notebooks/yolov5/runs/train/exp13/weights/best.pt', '/content/drive/MyDrive/Final-Year-Project-AI/object-sense-avoid-ai/notebooks/yolov5/runs/train/exp9/weights/best.pt'], batch_size=32, imgsz=1600, conf_thres=0.001, iou_thres=0.6, task=val, device=, workers=8, single_cls=False, augment=True, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=../runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 8b9537a torch 1.10.0+cu111 CUDA:0 (Tesla K80, 11441MiB)

Fusing layers... 
Model Summary: 213 layers, 7018216 parameters, 0 gradients, 15.8 GFLOPs
Fusing layers... 
Model Summary: 213 layers, 7018216 parameters, 0 gradients, 15.8 GFLOPs
Ensemble created with ['/content/drive/MyDrive/Final-Year-Project-AI/object-sense-avoid-ai/notebooks/yolov5/runs/train/ex

In [ ]:
# Inference
results = model(img, augment=True)  # TTA inference

FileNotFoundError: ignored

In [ ]:
# Print out the prediction
results.print()  
# results.save()  

# Return the Tensorflow format
results.xyxy

# Show the labeled image
results.show()

# Return the prediction dataframe
results.pandas().xyxy

In [ ]:
Image("./runs/detect/run/15553341149037764548af6dc9c2d794ea683e9554a0e728ca3.png")